In [ ]:
import estival

In [ ]:
from summer2.extras.test_models import sir

In [ ]:
m = sir()

In [ ]:
from numpyro import distributions as dist
import numpyro

In [ ]:
import numpy as np
from jax import numpy as jnp
import pandas as pd

In [ ]:
defp = m.get_default_parameters() 
mr = m.get_runner(defp, list(defp))#,solver="diffrax", solver_args={"diffeqsolve": {"max_steps": 4096, "throw": False}})#iffrax.DirectAdjoint()})
target = mr.function(defp)["derived_outputs"]["notifications"]
pd.Series(target).plot()

In [ ]:
target = pd.Series(jnp.exp(jnp.log(target) + np.random.normal(0.0,0.1,size=target.shape)))
target.plot()

In [ ]:
from estival.model import BayesianCompartmentalModel
from estival import targets as est, priors as esp

In [ ]:
bcm = BayesianCompartmentalModel(m, defp, [esp.UniformPrior(k, (0.0,1.0)) for k in defp], [est.NormalTarget("notifications", target, 1.)])

In [ ]:
def nmodel():
    sampled = {k:numpyro.sample(k, dist.Uniform(0.0,1.0)) for k in defp}

    ll = numpyro.factor("ll", bcm.loglikelihood(**sampled))

    #modelled = mr.function(sampled)["derived_outputs"]["notifications"]

    #lmodelled = jnp.log(modelled)    
    #dispersion = numpyro.sample("dispersion", dist.HalfNormal(1.0))

    #like = dist.Normal(lmodelled, dispersion).log_prob(jnp.log(target[30:60])).mean()
    #numpyro.factor("notifications_ll", like)


In [ ]:
from numpyro import infer
import jax
from jax import random
rkey = random.PRNGKey

In [ ]:
rng_key = rkey(0)

In [ ]:
init_params, potential_fn_gen, postprocess_fn, _ = infer.util.initialize_model(rkey(0), nmodel, dynamic_args=False)

In [ ]:
init_params.z

In [ ]:
logdensity_fn = lambda position: -potential_fn_gen(position)
initial_position = init_params.z

In [ ]:
logdensity_fn(initial_position)

In [ ]:
import blackjax

pf = blackjax.pathfinder(logdensity_fn)
rng_key, approx_key = jax.random.split(rng_key)
state, _ = pf.approximate(approx_key, initial_position, ftol=1e-4)

In [ ]:
from plotly import express as px

In [ ]:
# NUTS INIT

In [ ]:
adapt = blackjax.pathfinder_adaptation(blackjax.nuts, logdensity_fn)
rng_key, sample_key = jax.random.split(rng_key)
(state, parameters), info = adapt.run(sample_key, initial_position, 400)

In [ ]:
def inference_loop(rng_key, kernel, initial_state, num_samples):
    @jax.jit
    def one_step(state, rng_key):
        state, info = kernel(rng_key, state)
        return state, (state, info)

    keys = jax.random.split(rng_key, num_samples)
    _,states = jax.lax.scan(one_step, initial_state, keys)
    return states



In [ ]:
n_chains = 4
num_samples = 1000

nuts = blackjax.nuts(logdensity_fn, **parameters)
init0 = nuts.init(state.position)
rng_key, sample_key = jax.random.split(rng_key)
lkeys = jax.random.split(rng_key, n_chains)

def iloop(lkey):
    return inference_loop(lkey, nuts.step, init0, num_samples)


pstates = jax.pmap(iloop, in_axes=(0,))(lkeys)

In [ ]:
samples = postprocess_fn(pstates[0].position)

In [ ]:
import arviz as az

In [ ]:
idata = az.from_dict(postprocess_fn(pstates[0].position))

In [ ]:
az.summary(idata)

In [ ]:
from estival.sampling import tools as esamp

In [ ]:
samples = esamp.xarray_to_sampleiterator(az.extract(idata,num_samples=2000))

In [ ]:
az.plot_trace(idata,compact=False)

In [ ]:
# /NUTS INIT

In [ ]:
#numpyro.diagnostics.print_summary(samples, prob=0.89, group_by_chain=False)

In [ ]:
ssamples = esamp.xarray_to_sampleiterator(az.extract(idata,num_samples=2000))

In [ ]:
sres = [np.array(mr.function(s)["derived_outputs"]["notifications"]) for _, s in ssamples.iterrows()]

In [ ]:
q = np.quantile(np.stack(sres), (0.01,0.1,0.5,0.9,0.99),axis=0)

In [ ]:
pd.DataFrame((q)).T.plot(legend='')
pd.Series((target)).plot(style='.',legend='',color='black',)

In [ ]:
# Or do the whole thing just using numpyro! 

In [ ]:
from numpyro import infer

In [ ]:
kernel = infer.NUTS(nmodel)
mcmc = infer.MCMC(kernel, num_warmup=1000, num_chains=4, num_samples=1000)

In [ ]:
from jax import random

In [ ]:
mcmc.run(random.PRNGKey(0))

In [ ]:
idata = az.from_numpyro(mcmc)

In [ ]:
az.summary(idata)